#### MongoDB에 데이터 insert

필요 라이브러리 사전 설치

In [ ]:
#%pip install wheel
#%pip install pipwin
#%pip install geopandas

  Using cached geopandas-1.0.1-py3-none-any.whl.metadata (2.2 kB)
  Using cached pyogrio-0.10.0-cp311-cp311-win_amd64.whl.metadata (5.6 kB)
Using cached geopandas-1.0.1-py3-none-any.whl (323 kB)
Using cached pyogrio-0.10.0-cp311-cp311-win_amd64.whl (16.2 MB)
Note: you may need to restart the kernel to use updated packages.


In [22]:
import os
import geopandas as gpd
import pandas as pd
from fiona.crs import from_epsg
from shapely.geometry import Point
from pymongo import MongoClient

In [8]:
#os.chdir('../')
os.getcwd()

'c:\\Users\\11\\Desktop\\LCG\\Programming\\project\\wedrive'

In [9]:
# 디렉터리 경로
path_directory=os.getcwd()+'/data/od_uuid/2024/'
# 센서스 데이터
location_label_path=os.getcwd()+'/data/행정구역구분/BND_SIGUNGU_PG.shp'

# 데이터프레임 칼럼 지정
column_name=['uuid','time_begin','time_end','origin_lat','origin_lng','destination_lat','destination_lng','distance','heart','rank']

In [10]:
# 행정동 데이터 좌표계 변환
location_label=gpd.read_file(location_label_path, encoding='euc-kr')
location_label.to_crs(epsg=4326, inplace=True)
location_label

,BASE_DATE,SIGUNGU_NM,SIGUNGU_CD,geometry
0,20230701,종로구,11010,"POLYGON ((126.98 37.631, 126.97 37.63, 126.97 ..."
1,20230701,중구,11020,"POLYGON ((127.02 37.572, 127.02 37.572, 127.02..."
2,20230701,용산구,11030,"POLYGON ((126.97 37.555, 126.97 37.555, 126.97..."
3,20230701,성동구,11040,"POLYGON ((127.04 37.573, 127.04 37.573, 127.04..."
4,20230701,광진구,11050,"POLYGON ((127.1 37.572, 127.1 37.572, 127.1 37..."
...,...,...,...,...
245,20230701,함양군,38580,"POLYGON ((127.7 35.758, 127.7 35.758, 127.7 35..."
246,20230701,거창군,38590,"POLYGON ((127.88 35.906, 127.88 35.905, 127.88..."
247,20230701,합천군,38600,"POLYGON ((128.1 35.831, 128.1 35.83, 128.1 35...."
248,20230701,제주시,39010,"MULTIPOLYGON (((126.17 33.283, 126.17 33.283, ..."


In [19]:
# raw data에서 칼럼 추가하기
def Make_column(csv_file):
    raw_data=pd.read_csv(csv_file)
    data=raw_data.values.tolist()
    col_data=pd.DataFrame(data, columns=column_name)
    return col_data

# 데이터프레임에 destination_loc 칼럼 추가
def Make_column_destination_loc(csv_file):
    csv_file=Make_column(csv_file)

    #종료 위치 Point 객체
    end_geometry_Point=gpd.points_from_xy(csv_file['destination_lng'], csv_file['destination_lat'])
    coulumns = ['destination_lat', 'destination_lng']
    destination_gdf_data = gpd.GeoDataFrame(csv_file[coulumns], geometry=end_geometry_Point, crs=from_epsg(4326))
    destination_gdf_data['end_geometry'] = destination_gdf_data.apply(lambda row : Point([row['destination_lng'], row['destination_lat']]), axis=1)
    
    # Point 객체에 지역을 매핑시킨 데이터프레임 생성
    df=gpd.sjoin(destination_gdf_data, location_label)
    df.drop(columns=['index_right'], axis=1, inplace=True)
    #최종 데이터 프레임 생성
    csv_file['destination_area']=df['SIGUNGU_NM']
    
    return csv_file


In [20]:
# 테스트 b1734fe12445429483e03a242818091d.csv
filename=os.getcwd()+'/data/od_uuid/2024/b1734fe12445429483e03a242818091d.csv'
test_dataframe=Make_column_destination_loc(filename)
test_dataframe

C:\Users\11\AppData\Local\Temp\ipykernel_13448\2528327858.py:3: FionaDeprecationWarning: This function will be removed in version 2.0. Please use CRS.from_epsg() instead.
  test_dataframe=Make_column_destination_loc(filename)


,uuid,time_begin,time_end,origin_lat,origin_lng,destination_lat,destination_lng,distance,heart,rank,destination_area
0,b1734fe12445429483e03a242818091d,2024-01-04 10:16:33,2024-01-04 12:22:24,37.628888,127.071325,37.757372,127.033117,36800,43,S,의정부시
1,b1734fe12445429483e03a242818091d,2024-01-04 22:59:43,2024-01-05 00:07:14,37.756905,127.032728,37.541590,126.885623,34400,41,S,영등포구
2,b1734fe12445429483e03a242818091d,2024-01-05 01:15:14,2024-01-05 02:31:40,37.480977,126.982068,37.514648,127.099409,26816,26,B,송파구
3,b1734fe12445429483e03a242818091d,2024-01-05 02:33:22,2024-01-05 02:45:39,37.515498,127.098661,37.540133,127.070850,3917,3,C,광진구
4,b1734fe12445429483e03a242818091d,2024-01-05 02:45:56,2024-01-05 03:39:55,37.540133,127.070850,37.757081,127.032092,30239,33,A,의정부시
5,b1734fe12445429483e03a242818091d,2024-01-05 07:46:39,2024-01-05 08:11:39,37.757152,127.032200,37.746974,127.053048,3980,4,S,의정부시


In [26]:
# 데이터프레임에서 데이터를 추출하여 dictionary 형태로 변경한 후, MongoDB에 데이터 삽입
def dataframe_to_dict(dataframe):
    cl.insert_many(dataframe.to_dict('records'))



In [27]:
#테스트
dataframe_to_dict(test_dataframe)